# Auxiliary IYPNB for GPU Access

### Set-Up

In [ ]:
%cd "/content"

/content


In [ ]:
!git clone https://github.com/joaquintb/vdm_prueba.git

Cloning into 'vdm_prueba'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (401/401), done.
remote: Compressing objects: 100% (345/345), done.
remote: Total 401 (delta 77), reused 363 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (401/401), 5.70 MiB | 39.45 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [ ]:
%cd "vdm_prueba"

/content/vdm_prueba


In [ ]:
!pip -q install -U \
  diffusers==0.36.0 \
  accelerate==1.12.0 \
  peft==0.18.1 \
  transformers==4.57.6 \
  open-clip-torch==2.23.0 \
  medmnist==3.0.2 \
  pandas scikit-learn tqdm safetensors

In [ ]:
!python -c "import torch; print('CUDA available:', torch.cuda.is_available()); print('Num GPUs:', torch.cuda.device_count()); print('GPU name:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)"

CUDA available: True
Num GPUs: 1
GPU name: Tesla T4


In [ ]:
!python -c "import accelerate, diffusers, transformers, peft, datasets; print('OK'); print('diffusers', diffusers.__version__); print('transformers', transformers.__version__); print('peft', peft.__version__)"

2026-01-18 13:50:56.914254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768744256.938405    1186 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768744256.944808    1186 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768744256.960160    1186 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768744256.960186    1186 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768744256.960193    1186 computation_placer.cc:177] computation placer alr

### Prepare and Label

In [ ]:
!python -m src.dataset.prepare_and_label

/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:49: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
100% 4.17M/4.17M [00:02<00:00, 2.02MB/s]
Labelling train split:   0% 0/148 [00:00<?, ?batch/s]
open_clip_pytorch_model.bin:   0% 0.00/784M [00:00<?, ?B/s]
open_clip_pytorch_model.bin:   6% 46.1M/784M [00:02<00:46, 15.7MB/s]
open_clip_pytorch_model.bin:  14% 113M/784M [00:03<00:14, 44.8MB/s] 
open_clip_pytorch_model.bin:  23% 180M/784M [00:03<00:07, 75.8MB/s]
open_clip_pytorch_model.bin:  32% 247M/784M [00:03<00:04, 113MB/s] 
open_clip_pytorch_model.bin:  49% 381M/784M [00:03<00:02, 200MB/s]
open_clip_pytorch_model.bin:  57% 448M/784M [00:04<00:01, 215MB/s]
open_clip_pytorch_model.bin:  66% 515M/784M [00:04<00:01, 238MB/s]
open_clip_pytorch_model.bin:  74% 583M/784M [00:04<00:00, 237MB/s]
open_clip_pytorch_model.bin:

### LoRa Fine-Tuning

In [ ]:
!python -m src.generation.lora_fine_tuning


Running LoRA fine-tuning (official Diffusers script):

accelerate launch third_party/diffusers/train_text_to_image_lora.py --pretrained_model_name_or_path runwayml/stable-diffusion-v1-5 --train_data_dir data/subset --resolution 256 --train_batch_size 1 --gradient_accumulation_steps 4 --learning_rate 0.0001 --max_train_steps 200 --rank 8 --seed 42 --output_dir results/lora/pneumonia_lora --mixed_precision fp16

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2026-01-18 11:39:09.246792: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been reg

### LCM

In [ ]:
!python -m src.generation.lcm_generate

2026-01-18 12:02:37.993169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768737758.017845   10912 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768737758.028473   10912 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768737758.060367   10912 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768737758.060393   10912 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768737758.060399   10912 computation_placer.cc:177] computation placer alr

### Running the Pipeline

In [ ]:
!python -m src.pipeline --force

/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:49: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
2026-01-18 13:52:18.464298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768744338.482381    1561 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768744338.488254    1561 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768744338.503040    1561 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target mor

### Pushing Changes

In [ ]:
!git config --global user.name "Joaquin Torres"
!git config --global user.email "joaquintobrw@gmail.com"

In [ ]:
!git add .
!git commit -m "Run whole pipeline in colab for the first time"

[main ea82d57] Run whole pipeline in colab for the first time
 410 files changed, 368 insertions(+), 59 deletions(-)
 rewrite data/subset/images/img_00122.png (99%)
 rewrite data/subset/images/img_00123.png (99%)
 copy data/subset/images/{img_00122.png => img_00124.png} (100%)
 rewrite data/subset/images/img_00125.png (99%)
 rewrite data/subset/images/img_00126.png (99%)
 copy data/subset/images/{img_00123.png => img_00127.png} (100%)
 rename data/subset/images/{img_00124.png => img_00128.png} (100%)
 rewrite data/subset/images/img_00129.png (99%)
 rewrite data/subset/images/img_00130.png (99%)
 copy data/subset/images/{img_00125.png => img_00131.png} (100%)
 rewrite data/subset/images/img_00132.png (99%)
 copy data/subset/images/{img_00126.png => img_00133.png} (100%)
 rewrite data/subset/images/img_00134.png (99%)
 rewrite data/subset/images/img_00135.png (99%)
 rewrite data/subset/images/img_00136.png (99%)
 rename data/subset/images/{img_00127.png => img_00137.png} (100%)
 rewrite 

In [ ]:
import getpass, os
token = getpass.getpass("Paste your GitHub PAT (input hidden): ")
os.environ["GITHUB_TOKEN"] = token

Paste your GitHub PAT (input hidden): ··········


In [ ]:
!git remote -v
!git remote set-url origin https://$GITHUB_TOKEN@github.com/joaquintb/vdm_prueba.git
!git push

origin	https://github.com/joaquintb/vdm_prueba.git (fetch)
origin	https://github.com/joaquintb/vdm_prueba.git (push)
Enumerating objects: 396, done.
Counting objects: 100% (396/396), done.
Delta compression using up to 2 threads
Compressing objects: 100% (338/338), done.
Writing objects: 100% (339/339), 5.06 MiB | 12.83 MiB/s, done.
Total 339 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/joaquintb/vdm_prueba.git
   6fa1439..ea82d57  main -> main


### Pulling

In [ ]:
!git pull

remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13 (delta 8), reused 10 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (13/13), 2.49 KiB | 510.00 KiB/s, done.
From https://github.com/joaquintb/vdm_prueba
   e03875a..7eec749  main       -> origin/main
Updating e03875a..7eec749
Fast-forward
 .gitignore                     |   2 +
 src/generation/lcm_generate.py | 111 ++++++++++++++++++++++++++++++-----------
 2 files changed, 83 insertions(+), 30 deletions(-)
